In [1]:
from MapXtremePCIC import MapXtremePCIC
import pandas as pd
from numpy import ma
from sklearn.decomposition import pca
from scipy import interpolate
import numpy as np

In [2]:
PATH = './data/pr_24hr_50yrs/'
MapXtreme = MapXtremePCIC('110m', 'eof', PATH, 'pr')

In [3]:
df_obs = MapXtreme.get_pseudo_obs()
df_model = MapXtreme.standard_matrix(MapXtreme.ensemble_mean(MapXtreme.get_df())).dropna()
df = MapXtreme.get_df()

         areas       rlat       rlon
0  1772.265835 -27.279999 -32.560013
1  1780.387597 -27.279999 -32.119995
2  1788.304531 -27.279999 -31.679993
3  1796.161150 -27.279999 -31.239990
4  1804.299786 -27.279999 -30.800003
   run0  run1  run2  run3  run4  run5  run6  run7  run8  run9  ...    run25  \
0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...      NaN   
1   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...      NaN   
2   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...      NaN   
3   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...      NaN   
4   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...      NaN   

   run26  run27  run28  run29  run30  run31  run32  run33  run34  
0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  
1    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  
2    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  
3    NaN    NaN    NaN    NaN    Na

In [9]:
n_full, p_full, X_full = MapXtreme.get_arr(df_model)
n_field, p_field, X_field = MapXtreme.get_arr(df_obs)

skpca = pca.PCA(0.95)
skpca.fit_transform(X_full) #_transform
EOFs = skpca.components_.T

df_model['run_full_eof'] = EOFs[:, 0]

df_new = MapXtreme.combine_model_obs(df_model, df)
#pd.concat([df_model, df], axis = 1)
z = np.reshape(df_new['run_full_eof'].interpolate(method='nearest').fillna(0.0).values, (df_new['rlat'].unique().shape[0], df_new['rlon'].unique().shape[0]))

new_grid=interpolate.interp2d(df_new['rlon'].unique(), df_new['rlat'].unique(), z, kind = 'cubic')